<a href="https://colab.research.google.com/github/masonnlp/bioasq_qa_module/blob/master/bioasq_qa_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/My Drive/bert/


In [ ]:
%pip install tensorflow-gpu==1.13.1
%pip install dicttoxml

In [ ]:
from bs4 import BeautifulSoup as bs
import json
from json import loads
from dicttoxml import dicttoxml
content = []
paragraphs=[]
data ={}
# Read the XML file and preprocess input
with open("/content/test_input.xml", "r") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
   
    content = "".join(content)
    bs_content = bs(content, "lxml")
    result =(bs_content.find_all("q"))
    result1 = bs_content.find_all("summary")
    result2 = bs_content.find_all("id")
    #print(result2)
    for item,item1,item2 in zip(result,result1,result2):
      q=item.get_text(strip=True)
      c =item1.get_text(strip=True)
      i = item2.get_text(strip=True)
      qa = [{'id': i, "question": q}]
      dict1={'qas':qa,'context':c}
      paragraphs.append(dict1)  
data['data'] = [{"paragraphs": paragraphs}]
print(data)

with open('data3.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)
    outfile.close()
#call biobert for answering the questions
!python run_squad.py \
  --vocab_file=/content/vocab.txt \
  --bert_config_file=/content/bert_config.json \
  --init_checkpoint=../BERT-pubmed-1000000-SQuAD/model.ckpt-14599 \
  --do_train=False \
  --max_query_length=30  \
  --do_predict=True \
  --predict_file=data2.json \
  --predict_batch_size=8 \
  --n_best_size=3 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=/content/output/
#convert biobert's output to xml
json_file_path = "/content/output/predictions.json"

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())
xml = dicttoxml(contents)

with open('bert_predictions2.xml', 'wb') as outfile:
    outfile.write(xml)
    outfile.close()

